## Limpieza y Preparacion De Datos.

Durante el proceso de análisis y modelado de datos, se dedica una cantidad significativa de tiempo a la preparación de datos: **carga, limpieza, transformación y reordenamiento.** A menudo se informa que tales tareas ocupan el **80% o más del tiempo de un analista.** A veces, la forma en que se almacenan los datos en archivos o bases de datos no está en el formato correcto para una tarea en particular. Muchos investigadores optan por realizar un procesamiento **ad hoc** de datos de una forma a otra utilizando un lenguaje de programación de propósito general, como Python, Perl, R o Java, o herramientas de procesamiento de texto Unix como sed o awk. Afortunadamente, pandas, junto con las funciones integradas del lenguaje Python, le proporciona un conjunto de herramientas rápido, flexible y de alto nivel que le permite manipular los datos en la forma correcta.

Si identifica un tipo de manipulación de datos que no se encuentra en ninguna parte de este libro o de la biblioteca de pandas, no dude en compartir su caso de uso en una de las listas de correo de Python o en el sitio de GitHub de pandas. De hecho, gran parte del diseño y la implementación de pandas ha sido impulsado por las necesidades de las aplicaciones del mundo real.

En este capítulo, analizaremos las herramientas para **datos faltantes, datos duplicados, manipulación de cadenas y algunas otras transformaciones de datos analíticos.**


## Lidiando con Datos Faltantes.

La falta de datos ocurre comúnmente en muchas aplicaciones de análisis de datos. Uno de los objetivos de  pandas es hacer que **trabajar con los datos faltantes sea lo más sencillo posible.** Por ejemplo, **todas las estadísticas descriptivas de los objetos pandas excluyen los datos faltantes de forma predeterminada.**

La forma en que se representan los datos faltantes en los objetos pandas es algo imperfecta, pero es funcional para muchos usuarios. Para los **datos numéricos**,  pandas usa el valor de punto flotante **NaN** (No es un número) para representar los datos faltantes. A esto lo llamamos un valor **centinela que se puede detectar fácilmente**:


In [1]:
import pandas as pd
import numpy as np

In [2]:
s_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
s_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
s_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

En pandas, hemos adoptado una convención utilizada en el lenguaje de programación R al referirnos a los datos faltantes como **NA**, que significa **no disponible.** En las aplicaciones de estadística, los datos de **NA pueden ser datos que no existen o que existen pero que no se observaron (a través de problemas con la recopilación de datos, por ejemplo).** Cuando se limpian los datos para su análisis, a menudo es **importante realizar un análisis de los datos faltantes para identificar problemas de recopilación de datos o posibles sesgos en los datos causados por datos faltantes.**

El valor integrado de Python None también se trata como NA en las matrices de objetos:

In [5]:
s_data[0] = None

In [6]:
s_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Se está trabajando en el proyecto pandas para mejorar los detalles internos de cómo se manejan los datos faltantes, pero las funciones de la API del usuario, como pandas.isnull, abstraen muchos de los detalles molestos.

 *  dropna - Filtra etiquetas de eje en función de si los valores de cada etiqueta tienen datos faltantes, con umbrales variables de cuántos datos faltantes tolerar.

 * fillna -Completa los datos faltantes con algún valor o utilizando un método de interpolación como 'ffill' y 'bfill'.

 *  isnull - Retorna un valor booleano indicando que valores faltan

 * notnull negacion de isnull

## Filtrar datos faltantes


Hay algunas formas de filtrar los datos faltantes. Si bien siempre tiene la opción de hacerlo a mano usando pandas.isnull y la indexación booleana, el dropna puede ser útil. En una serie, devuelve la serie solo con los datos no nulos y los valores de índice:

In [7]:
from numpy import  nan as NA

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [9]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

es equivalente a:


In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

Con los objetos DataFrame, las cosas son un poco más complejas. Es posible que desee eliminar filas o columnas que sean todas NA o solo aquellas que contengan NA. dropna por defecto elimina cualquier fila que contenga un valor faltante:

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
limpio = data.dropna()
limpio

,0,1,2
0,1.0,6.5,3.0


In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass axis=1:


In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1,how='all') # eliminamos columna de puros NA

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Una forma relacionada de filtrar filas de DataFrame tiende a referirse a datos de series de tiempo. Suponga que desea mantener **solo filas que contengan un cierto número de observaciones.** Puede indicar esto con el argumento thresh:

In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4,1] = NA
df.iloc[:2,2] = NA
df

,0,1,2
0,1.267910,NaN,NaN
1,0.843434,NaN,NaN
2,-0.541975,NaN,-0.580101
3,0.464106,NaN,0.740549
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


In [17]:
df.dropna()

,0,1,2
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


In [18]:
df.dropna(thresh=2)

,0,1,2
2,-0.541975,NaN,-0.580101
3,0.464106,NaN,0.740549
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


## Completar los datos que faltan

En lugar de filtrar los datos faltantes (y posiblemente descartar otros datos junto con ellos), es posible que desee rellenar los "huecos" de varias formas. Para la mayoría de los propósitos, el método fillna es la función fillna . Llamar a fillna con una constante reemplaza los valores faltantes con ese valor:

In [19]:
df.fillna(0)

,0,1,2
0,1.267910,0.000000,0.000000
1,0.843434,0.000000,0.000000
2,-0.541975,0.000000,-0.580101
3,0.464106,0.000000,0.740549
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


In [20]:
df.fillna({1:0.5,2:0})

,0,1,2
0,1.267910,0.500000,0.000000
1,0.843434,0.500000,0.000000
2,-0.541975,0.500000,-0.580101
3,0.464106,0.500000,0.740549
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


In [21]:
# modificamos el objeto existente 
_ = df.fillna(0,inplace=True)

df

,0,1,2
0,1.267910,0.000000,0.000000
1,0.843434,0.000000,0.000000
2,-0.541975,0.000000,-0.580101
3,0.464106,0.000000,0.740549
4,1.722554,0.698796,-2.426928
5,0.603774,-0.563012,0.539334
6,-1.336332,0.396301,0.044942


In [22]:
# rellenamos datos faltantes con algun estadistico
data = pd.Series([1.,NA,3.5,NA,7])

data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Transformacion de Datos.

Hasta ahora, en este capítulo, nos hemos ocupado de reorganizar los datos. El **filtrado, la limpieza y otras transformaciones son otra clase de operaciones importantes.**


## Removiendo Valores Duplicados

Se pueden encontrar filas duplicadas en un DataFrame por diversas razones. Aquí hay un ejemplo:

In [23]:
data = pd.DataFrame(
    {'k1': ['one', 'two'] * 3 + ['two'],
     'k2': [1, 1, 2, 3, 3, 4, 4]}
)

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [24]:
# vemos si existen valores duplicados

data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [25]:
# eliminamos valores duplicados
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Ambos métodos de forma predeterminada consideran **todas las columnas; como alternativa,** puede especificar cualquier subconjunto de ellos para detectar duplicados. Supongamos que tenemos una columna adicional de valores y queremos filtrar los duplicados solo en función de la columna 'k1':

In [26]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [27]:
# eliminamos los valores duplicados de la columna k1
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated y drop_duplicates conservan de forma predeterminada la primera combinación de valores observados. Pasando keep = 'last' devolverá el último:

In [28]:
data.drop_duplicates(['k1','k2'],keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transformar datos usando mapping

Para muchos conjuntos de datos, es posible que desee realizar alguna transformación en función de los valores en una matriz, serie o columna en un DataFrame. Considere los siguientes datos hipotéticos recopilados sobre varios tipos de carne:

In [29]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                                  'Pastrami', 'corned beef', 'Bacon',
                                  'pastrami', 'honey ham', 'nova lox'],
                         'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suponga que desea **agregar una columna que indique el tipo de animal del que proviene cada alimento.** Escribamos un mapeo de cada tipo de carne distinto al tipo de animal:

In [30]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

El método  map en una serie acepta una función o un objeto similar a un dictado que contiene un mapa, pero aquí tenemos un pequeño problema en el que algunas de las carnes están en **mayúsculas y otras no.** Por lo tanto, necesitamos convertir cada valor a minúsculas usando el método str.lower Series:

In [31]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [32]:
# creamos una columna nueva con el tipo de nimal del cual proviene ese producto
data['animal'] = lowercased.map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [33]:
# podemos hacer esto mismo aplicando una funcion lambda
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## Reemplazo de valores


Completar los datos faltantes con el método fillna es un caso especial de reemplazo de valor más general. Como ya ha visto, el mapa se puede utilizar para modificar un subconjunto de valores en un objeto, pero reemplazar proporciona una forma más sencilla y flexible de hacerlo. Consideremos esta serie:

In [34]:
data = pd.Series([1.,-999,.2,-1000.,.3])
data

0       1.0
1    -999.0
2       0.2
3   -1000.0
4       0.3
dtype: float64

Los valores -999 pueden ser valores centinela para los datos faltantes. Para reemplazar estos con valores NA que los pandas entienden, podemos usar reemplazar, produciendo una nueva Serie.

In [35]:
data.replace(-999,np.nan)

0       1.0
1       NaN
2       0.2
3   -1000.0
4       0.3
dtype: float64

In [36]:
# remplazamos varios valores al mismo tiempo
data.replace([-999,-1000],np.nan)

0    1.0
1    NaN
2    0.2
3    NaN
4    0.3
dtype: float64

In [37]:
# remplazamos diferentes valores con diferentes parametros
data.replace([-1000,-999],[np.nan,0])

0    1.0
1    0.0
2    0.2
3    NaN
4    0.3
dtype: float64

In [38]:
# tambein puede ser un diccionario
data.replace({-999:0,-1000:np.nan})

0    1.0
1    0.0
2    0.2
3    NaN
4    0.3
dtype: float64

## Cambiar el nombre de los índices de eje.

Al igual que los valores en una serie, las etiquetas de los ejes se pueden transformar de manera similar mediante una función o mapeo de alguna forma para producir objetos nuevos con etiquetas diferentes. También puede modificar los ejes in place sin crear una nueva estructura de datos. Aquí tienes un ejemplo simple:

In [39]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
index=['Ohio', 'Colorado', 'New York'],
columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [40]:
x = lambda x: x[:4].upper()
data.index.map(x)
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [41]:
data.index = data.index.map(x)

data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [42]:
data.rename(index=str.title,columns=str.upper)


,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


## Discretización y Binning

Los datos continuos a menudo se discretizan o se separan en **"contenedores" para su análisis.** Suponga que tiene datos sobre un grupo de personas en un estudio y desea agruparlos en grupos de edad discretos:

In [43]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Dividamos estos en contenedores de 18 a 25, 26 a 35, 36 a 60 y, finalmente, 61 años o más. Para hacerlo, debes usar cut, una función en pandas:

In [44]:
contenedores = [18,25,35,60,100]

cats = pd.cut(ages,contenedores) # agrupamos los datos de ages en el contenedor

cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [45]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

El objeto que devuelve pandas es un objeto categórico especial. El resultado que ve describe los contenedores calculados por pandas.cut. Puede tratarlo como una matriz de cadenas que indica el nombre del contenedor; Internamente, contiene una matriz de categorías que especifica los distintos nombres de categoría junto con un etiquetado para los datos de edades en el atributo de códigos:

In [46]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [47]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

De acuerdo con la notación matemática para intervalos, un paréntesis significa que el lado está abierto, mientras que el corchete significa que está cerrado (inclusive). Puede cambiar qué lado está cerrado pasando right = False:

In [48]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [49]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, contenedores, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

## Detectar y filtrar valores atípicos.

Filtrar o transformar valores atípicos es en gran medida una cuestión de aplicar operaciones de matriz. Considere un DataFrame con algunos datos distribuidos normalmente:

In [50]:
data = pd.DataFrame(np.random.randn(1000,4))

data

,0,1,2,3
0,0.388468,1.218930,-2.272606,-0.563985
1,0.642866,0.564874,-0.913024,-1.025611
2,0.566766,-0.997671,0.589881,0.282793
3,-0.597929,1.620805,-0.619296,0.513623
4,-0.716644,0.228229,-0.959649,-0.435343
...,...,...,...,...
995,-0.417408,-0.361139,1.276672,-0.158543
996,-0.730099,0.881668,0.934171,0.863943
997,-1.370391,0.320331,-0.103489,-0.641419
998,1.793079,0.146647,1.196727,0.904867


Suponga que desea encontrar valores en una de las columnas que excedan 3 en valor absoluto:

In [51]:
col = data[2]
col[np.abs(col) > 3]

668    3.297057
676    3.166405
Name: 2, dtype: float64

Para seleccionar todas las filas que tengan un valor superior a 3 o –3, puede utilizar el método any en un DataFrame booleano:

In [52]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
298,-1.117800,1.627931,-1.483209,-3.039036
520,0.642204,3.050089,-0.290626,1.479401
668,1.609745,-1.110378,3.297057,0.140162
674,1.786794,-2.038628,-1.247529,-3.125410
676,-1.120445,1.096405,3.166405,-0.531976
707,-3.072393,2.347324,-1.375020,0.120477
774,-0.324608,2.649255,-0.196881,-3.104602


## Permutación y muestreo aleatorio
Permutar (reordenar aleatoriamente) una Serie o las filas en un DataFrame es fácil de hacer usando la función numpy.random.permutation. Llamar a la permutación con la longitud del eje que desea permutar produce una matriz de enteros que indica el nuevo orden:

In [53]:
 df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
 df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [54]:
sampler = np.random.permutation(5)

In [55]:
sampler

array([4, 2, 0, 1, 3])

In [56]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15


## Indicador de cálculo / variables ficticias
Otro tipo de transformación para aplicaciones de aprendizaje automático o modelado estadístico es convertir una variable categórica en una matriz "ficticia" o "indicadora". Si una columna en un DataFrame tiene k valores distintos, derivaría una matriz o DataFrame con k columnas que contienen todos los 1 y 0. pandas tiene una función get_dummies para hacer esto, aunque diseñar una usted mismo no es difícil. Consideremos un ejemplo de DataFrame:

In [57]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [58]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


En algunos casos, es posible que desee agregar un prefijo a las columnas en el indicador DataFrame, que luego se puede fusionar con los otros datos. get_dummies tiene un argumento de prefijo para hacer esto:

In [59]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)

In [60]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [62]:
students = {'Alice':'Fisica',
            'Bob':'Math',
            'Jane':'Stats',
            'Orreyli':'DS'}

s = pd.Series(students)
s

Alice      Fisica
Bob          Math
Jane        Stats
Orreyli        DS
dtype: object

In [64]:
s.iloc[3]

'DS'

In [65]:
s.loc['Bob']

'Math'

In [66]:
s[1]

'Math'

In [70]:
s[2:]

Jane       Stats
Orreyli       DS
dtype: object

In [72]:
class_code = {10:'Stats',
              7:'POO',
              10:'Data Wrangling',
              9:'English',
              10:'Big Data'}
s = pd.Series(class_code)
s

10    Big Data
7          POO
9      English
dtype: object

In [73]:
s[0]

KeyError: ignored

In [74]:
s.iloc[0]

'Big Data'

In [75]:
s.iloc[1]

'POO'

In [84]:
import random
n = 20
lista_aleatorios = [random.randint(10,90) for _ in range(n)]

s = pd.Series(lista_aleatorios)
s

0     26
1     27
2     82
3     74
4     59
5     31
6     56
7     25
8     72
9     60
10    45
11    49
12    81
13    61
14    40
15    85
16    81
17    88
18    67
19    21
dtype: int64

In [89]:
total = 0
for e in s:
  total+=e

print(total/len(s))

56.5


In [91]:
total = np.sum(s)
print(total/len(s))

56.5


In [92]:
numbs = pd.Series(np.random.randint(0,1000,10000))

numbs.head()

0     86
1     11
2    440
3    681
4    490
dtype: int64

In [93]:
len(numbs)

10000

In [97]:
%%timeit -n 100
# funcion magica 
total = 0
for number in numbs:
  total+=number
total/len(numbs)

100 loops, best of 3: 1.22 ms per loop


In [99]:
%%timeit -n 100
total = np.sum(numbs)
total/len(numbs)

100 loops, best of 3: 80.5 µs per loop


In [100]:
s.head()

0    26
1    27
2    82
3    74
4    59
dtype: int64

In [102]:
s+=2 
s.head(
    
)

0    30
1    31
2    86
3    78
4    63
dtype: int64

In [105]:
%%timeit -n 10
s = pd.Series(np.random.randint(0,1000,10000))

s+=2

10 loops, best of 3: 355 µs per loop


In [107]:
s = pd.Series([1,2,3],index=['math','stats','ds'])
s.loc['history'] = 2
s

math       1
stats      2
ds         3
history    2
dtype: int64

In [115]:
students = pd.Series({'fer':'stats',
                      'fras':'math',
                      'juana':'physiscs'})
kelly_classes = pd.Series(['Phylosophy','Art','Math'],index=['kelly','kelly','kelly'])
kelly_classes

kelly    Phylosophy
kelly           Art
kelly          Math
dtype: object

In [116]:
students

fer         stats
fras         math
juana    physiscs
dtype: object

In [117]:
all_students = students.append(kelly_classes)
all_students

fer           stats
fras           math
juana      physiscs
kelly    Phylosophy
kelly           Art
kelly          Math
dtype: object

In [118]:
students

fer         stats
fras         math
juana    physiscs
dtype: object

In [119]:
all_students.loc['kelly']

kelly    Phylosophy
kelly           Art
kelly          Math
dtype: object